Load the Data (2014-2016)

In [ ]:
import csv
import pandas as pd
import numpy as np
pd.set_option('display.max_rows',  1005)
pd.set_option('display.max_columns', 30)
column_names = ['C/A','Unit','SCP','Station','Line Name','Division','Date','Time','Description','Entries','Exits']
subway_data = pd.read_csv('subway_data_file.csv', nrows = 1000, encoding = 'utf-8', names= column_names)
farecard_data = pd.read_csv('Fare_Card_History_Beg2010.csv', nrows = 1000, encoding = 'utf-8')
#days_graph = pd.read_csv('day_rides_array_formatted.csv', encoding = 'utf-8')
top_20_routes_graph = pd.read_csv('C:/Users/Benjamin/Desktop/BigData1004/Term Project/Output_Files/Subway/subway_top_20_stations_farecard_formatted.csv', encoding = 'utf-8')
farecard_type_station_graph = pd.read_csv('C:/Users/Benjamin/Desktop/BigData1004/Term Project/Output_Files/Subway/subway_farecard_type_station.csv', encoding = 'utf-8')
#subway_data.to_csv('100k_subway_data.csv')
#farecard_data.to_csv('10k_farecard_data.csv')

In [ ]:
Most Popular End Stations, Most Popular Start Stations (Using Turnstile Dataset)

In [ ]:
kv_stations = []
init_kv_list = []
for x in range(0, len(subway_data)):
        if subway_data['Station'][x] == "Station":
            dummy = 0
        else: 
            # edit here
            key1 = subway_data['Station'][x]
            key5 = subway_data['Division'][x]
            key6 = subway_data['Line Name'][x]
            station_name_combo = ("%s%s%s" % (key1, key5, key6))
            ## sample station name clean replacement:
            #if station_name_combo == 'WHITEHALL STBMTR1':
            #        station_name_combo = station_name_combo.replace('WHITEHALL STBMTR1', '1 AVEBMTL')
            if subway_data['Date'][x] == "02/01/2014" and subway_data['Time'][x] == '00:00:00':
                #edit here
                init_station_name = ("%s&%s&%s&%s" % (station_name_combo, subway_data['C/A'][x], subway_data['Unit'][x], subway_data['SCP'][x]))
                init_enters  = subway_data['Entries'][x].replace(",","")
                init_exits = subway_data['Exits'][x].replace(",","")
                init_kv = ("%s,%s,%s" % (init_station_name, init_enters, init_exits))
                init_kv_list.append(init_kv)
            key2 = subway_data['C/A'][x] 
            key3 = subway_data['Unit'][x]
            key4 = subway_data['SCP'][x]
            value1 = subway_data['Entries'][x].replace(",", "")
            value2 = subway_data['Exits'][x].replace(",", "")
            #edit here
            kv = ("%s&%s&%s&%s\t%s,%s" % (station_name_combo, key2, key3, key4, value1, value2))
            kv_stations.append(kv)

kv_stations.sort()
keys_stations = []
enter_vals = []
exit_vals = []
enter_init_val_sum = 0
exit_init_val_sum = 0
for i in kv_stations:
    key, value = i.split('\t')
    keys_stations.append(key)
    enter, exit = value.split(',')
    enter = int(enter)
    exit = int(exit)
    enter_vals.append(enter)
    exit_vals.append(exit)

key_place = "hello"
first_key = 0
count_enters = 0
count_exits = 0
count_place = 0 
count_old_enters = 0
count_old_exits = 0
first_count_day = 0
new_kv_enter =[]
new_kv_exits = [] 

for j in keys_stations:
    if first_key == 0:
        key_place = j
        first_key = 1
    
    if j != key_place and first_key != 0:
        kv_enters = (count_enters, key_place)
        kv_exits = (count_exits, key_place)        
        new_kv_enter.append(kv_enters)
        new_kv_exits.append(kv_exits)
        key_place = j
        count_init_enters = enter_vals[count_place]
        count_init_exits = exit_vals[count_place]
        count_enters = 0
        count_exits = 0
        count_place += 1
    
    else:
        if first_count_day == 0:
            count_init_enters = enter_vals[count_place]
            count_init_exits = exit_vals[count_place]
            first_count_day = 1
            count_place += 1
        else:
            enters = enter_vals[count_place]
            exits = exit_vals[count_place]
            if enters >= count_init_enters:
                count_enters += (enter_vals[count_place] - count_init_enters)
            else:
                count_enters += enter_vals[count_place]
            if exits >= count_init_exits:
                count_exits += (exit_vals[count_place] - count_init_exits)
            else:
                count_exits += exit_vals[count_place]
            count_init_enters = enter_vals[count_place]
            count_init_exits = exit_vals[count_place]
            count_place += 1

kv_enters = (count_enters, key_place)
kv_exits = (count_exits, key_place)
new_kv_enter.append(kv_enters)
new_kv_exits.append(kv_exits)

stations_collapse_enter = []
stations_collapse_exit = []
    
def collapse(input_list, append_list):
    for z in input_list:
        station_name = z[1]
        station_sep = z[0]
        station_name, dummy1, dummy2, dummy3 = station_name.split("&")
        collapse_kv = (station_name, station_sep)
        append_list.append(collapse_kv) 
collapse(new_kv_enter, stations_collapse_enter)
collapse(new_kv_exits, stations_collapse_exit) 

stations_collapse_enter.sort()
stations_collapse_exit.sort()

final_kv_enter = [] 
final_kv_exits = []
def aggreg(input_list, append_list):
    first_key = 0 
    count_final = 0

    for p in input_list:
        if first_key == 0:
            key_place = p[0]
            first_key = 1
        
        if p[0] != key_place and first_key != 0: 
            final_kv = (count_final, key_place)
            append_list.append(final_kv)
            count_final = p[1]
            key_place = p[0]
        else:
            count_final += p[1]
    final_kv = (count_final, key_place)
    append_list.append(final_kv)

aggreg(stations_collapse_enter, final_kv_enter)
aggreg(stations_collapse_exit, final_kv_exits)

def ranking(array, order, top_or_bottom, start_or_end):
    array.sort(reverse = order)
    top_ten = []
    bottom_ten = []
    count_place = 1
    for x in range(0,1):
        top_ten.append(array[x])
    for y in top_ten:
        key_final = y[1]
        value_final = y[0]
        print("The %d %s popular %s station is: %s with %d trips" % (count_place, top_or_bottom, start_or_end, key_final, value_final))
        count_place +=1
ranking(final_kv_enter, True, "most", "start")
ranking(final_kv_exits, True, "most", "end") 
ranking(final_kv_enter, False, "least", "start")
ranking(final_kv_exits, False, "least",  "end")

Top 20 Stations Using Farecard History Dataset (complete and saved to output files)

In [ ]:
kv_stations = []
init_kv_list = []
for x in range(0, len(farecard_data)):
        if farecard_data['Station'][x] == "Station":
            dummy = 0
        else:
            key1 = farecard_data['Station'][x]
            #key2 = farecard_data['Remote Station ID'][x]         
            value1 = int(farecard_data['Full Fare'][x].replace(",", ""))
            value2 =  int(farecard_data['Senior Citizen / Disabled'][x].replace(",", ""))
            value3 = int(farecard_data['7 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
            value4 = int(farecard_data['30 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
            value5 = int(farecard_data['Joint Rail Road Ticket'][x].replace(",", ""))
            value6 =  int(farecard_data['7 Day Unlimited'][x].replace(",", ""))
            value7 =  int(farecard_data['30 Day Unlimited'][x].replace(",", ""))
            value8 =  farecard_data['14 Day Reduced Fare Media Unlimited'][x]
            value9 =  farecard_data['1 Day Unlimited'][x]
            value10 = farecard_data['14 Day Unlimited'][x]
            value11= int(farecard_data['Transit Check Metrocard'][x].replace(",", ""))
            value12 = int(farecard_data['LIB Special Senior'][x].replace(",", ""))
            value13 =  int(farecard_data['Rail Road Unlimited No Trade'][x].replace(",", ""))
            value14 = int(farecard_data['Transit Check Metrocard Annual Metrocard'][x].replace(",", ""))
            value15 =  int(farecard_data['Mail and Ride EZPass Express'][x].replace(",", ""))
            value16 =  int(farecard_data['Mail and Ride Unlimited'][x].replace(",", ""))
            sum_value  = value1 + value2 + value3 + value4 + value5 + value6 + value7 + value8 + value9 + value10 + value11 + value12 + value13 + value14 + value15 + value16
            
            kv = ("%s\t%s" % (key1, sum_value))
            kv_stations.append(kv)

kv_stations.sort()
keys_stations = []
enter_vals = []

for i in kv_stations:
    key, value = i.split('\t')
    keys_stations.append(key)
    value = int(value)
    enter_vals.append(value)

key_place = "hello"
first_key = 0
count_enters = 0
count_place = 0 
new_kv_enter =[] 

for j in keys_stations:
    if first_key == 0:
        key_place = j
        first_key = 1
    
    if j != key_place and first_key != 0:
        kv_enters = (count_enters, key_place)        
        new_kv_enter.append(kv_enters)
        key_place = j
        count_enters = enter_vals[count_place]
        count_place += 1
    
    else:
        count_enters += enter_vals[count_place]
        count_place += 1

kv_enters = (count_enters, key_place)
new_kv_enter.append(kv_enters)

stations_collapse_enter = []

    
def collapse(input_list, append_list):
    for z in input_list:
        station_name= z[1]
        station_sep = z[0]
        collapse_kv = (station_name, station_sep)
        append_list.append(collapse_kv) 
collapse(new_kv_enter, stations_collapse_enter)

stations_collapse_enter.sort()

final_kv_enter = [] 
def aggreg(input_list, append_list):
    first_key = 0 
    count_final = 0

    for p in input_list:
        if first_key == 0:
            key_place = p[0]
            first_key = 1
        
        if p[0] != key_place and first_key != 0: 
            final_kv = (count_final, key_place)
            append_list.append(final_kv)
            count_final = p[1]
            key_place = p[0]
        else:
            count_final += p[1]
    final_kv = (count_final, key_place)
    append_list.append(final_kv)

aggreg(stations_collapse_enter, final_kv_enter)


def ranking(array, order, top_or_bottom, start_or_end):
    array.sort(reverse = order)
    top_ten = []
    bottom_ten = []
    count_place = 1
    for x in range(0,1):
        top_ten.append(array[x])
    for y in top_ten:
        key_final = y[1]
        value_final = y[0]
        print("The %d %s popular %s station is: %s with %d trips" % (count_place, top_or_bottom, start_or_end, key_final, value_final))
        count_place +=1
ranking(final_kv_enter, True, "most", "start")
ranking(final_kv_enter, False, "least", "start")

Seasonality - Number of Trips By Month (complete and saved to output files)

In [ ]:
kv_months = []
for x in range(0, len(farecard_data)):
        #add year filter year  to get relevent years
        if int(farecard_data['To Date'][x][6:10]) >= 2017:
            dummy = 0
        elif int(farecard_data['To Date'][x][6:10]) == 2016 and int(farecard_data['To Date'][x][0:2]) >= 7:
            dummy = 0
        elif int(farecard_data['To Date'][x][6:10]) == 2010 and int(farecard_data['To Date'][x][0:2]) == 6:
            dummy = 0
        else:
            
            if int(farecard_data['To Date'][x][6:10]) <= 2012:
                key1 = "Pre-CitiBike"
            elif int(farecard_data['To Date'][x][6:10]) == 2013 and int(farecard_data['To Date'][x][0:2]) <= 6:
                key1 = "Pre-CitiBike"
            else:
                key1 = "With-CitiBike"
        key2 = farecard_data['To Date'][x][0:2]
        value1 = int(farecard_data['Full Fare'][x].replace(",", ""))
        value2 =  int(farecard_data['Senior Citizen / Disabled'][x].replace(",", ""))
        value3 = int(farecard_data['7 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
        value4 = int(farecard_data['30 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
        value5 = int(farecard_data['Joint Rail Road Ticket'][x].replace(",", ""))
        value6 =  int(farecard_data['7 Day Unlimited'][x].replace(",", ""))
        value7 =  int(farecard_data['30 Day Unlimited'][x].replace(",", ""))
        value8 =  farecard_data['14 Day Reduced Fare Media Unlimited'][x]
        value9 =  farecard_data['1 Day Unlimited'][x]
        value10 = farecard_data['14 Day Unlimited'][x]
        value11= int(farecard_data['Transit Check Metrocard'][x].replace(",", ""))
        value12 = int(farecard_data['LIB Special Senior'][x].replace(",", ""))
        value13 =  int(farecard_data['Rail Road Unlimited No Trade'][x].replace(",", ""))
        value14 = int(farecard_data['Transit Check Metrocard Annual Metrocard'][x].replace(",", ""))
        value15 =  int(farecard_data['Mail and Ride EZPass Express'][x].replace(",", ""))
        value16 =  int(farecard_data['Mail and Ride Unlimited'][x].replace(",", ""))
        sum_value  = value1 + value2 + value3 + value4 + value5 + value6 + value7 + value8 + value9 + value10 + value11 + value12 + value13 + value14 + value15 + value16
        kv = ("%s^%s\t%s" % (key1, key2, sum_value))
        kv_months.append(kv)
        
kv_months.sort()

keys_months = []
values = []
for i in kv_months:
    key, value = i.split('\t')
    keys_months.append(key)
    value = int(value)
    values.append(value)

key_place = 0
first_key = 0
count = 0
count_place = 0

for j in keys_months:
    if first_key == 0:
        key_place = j
        first_key = 1 
    
    if j != key_place and first_key != 0:
        year_type, month = key_place.split("^")
        avg_month = float(count / 3)
        print("The number of rides in %s %s were: %d, with an average of %.2f" % (month, year_type, count, avg_month))
        key_place = j
        count = values[count_place]
        count_place += 1
    else:
        count += values[count_place]
        count_place += 1
year_type, month = key_place.split("^")
avg_month = float(count / 3)
print("The number of rides in %s %s were: %d, with an average of %.2f" % (month, year_type, count, avg_month))

Seasonality - The Number of Trips in Good Weather Months (May - Sept) Vs Bad Weather Months (complete and saved to output files)

In [ ]:
kv_weather = []
for x in range(0, len(farecard_data)):
        #add year filter year  to get relevent years
        if farecard_data['To Date'][x][0:2] == "05" or farecard_data['To Date'][x][0:2] =="06" or farecard_data['To Date'][x][0:2] =="07" or farecard_data['To Date'][x][0:2] =="08" or farecard_data['To Date'][x][0:2] =="09":
            key = "Summer"
        else:
            key = "Winter"
        value1 = int(farecard_data['Full Fare'][x].replace(",", ""))
        value2 =  int(farecard_data['Senior Citizen / Disabled'][x].replace(",", ""))
        value3 = int(farecard_data['7 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
        value4 = int(farecard_data['30 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
        value5 = int(farecard_data['Joint Rail Road Ticket'][x].replace(",", ""))
        value6 =  int(farecard_data['7 Day Unlimited'][x].replace(",", ""))
        value7 =  int(farecard_data['30 Day Unlimited'][x].replace(",", ""))
        value8 =  farecard_data['14 Day Reduced Fare Media Unlimited'][x]
        value9 =  farecard_data['1 Day Unlimited'][x]
        value10 = farecard_data['14 Day Unlimited'][x]
        value11= int(farecard_data['Transit Check Metrocard'][x].replace(",", ""))
        value12 = int(farecard_data['LIB Special Senior'][x].replace(",", ""))
        value13 =  int(farecard_data['Rail Road Unlimited No Trade'][x].replace(",", ""))
        value14 = int(farecard_data['Transit Check Metrocard Annual Metrocard'][x].replace(",", ""))
        value15 =  int(farecard_data['Mail and Ride EZPass Express'][x].replace(",", ""))
        value16 =  int(farecard_data['Mail and Ride Unlimited'][x].replace(",", ""))
        sum_value  = value1 + value2 + value3 + value4 + value5 + value6 + value7 + value8 + value9 + value10 + value11 + value12 + value13 + value14 + value15 + value16
        kv = ("%s\t%s" % (key, sum_value))
        kv_weather.append(kv)
        
kv_weather.sort()

keys_weather = []
values = []
for i in kv_weather:
    key, value = i.split('\t')
    keys_weather.append(key)
    value = int(value)
    values.append(value)

key_place = 0
first_key = 0
count = 0
count_place = 0

for j in keys_weather:
    if first_key == 0:
        key_place = j
        first_key = 1 
    
    if j != key_place and first_key != 0:
        print("The number of rides in %s were: %d" % (key_place, count))
        key_place = j
        count = values[count_place]
        count_place += 1
    else:
        count += values[count_place]
        count_place += 1
print("The number of rides in %s were: %d" % (key_place, count))

Trips By Year (complete and save to output files)

In [ ]:
kv_years = []
for x in range(0, len(farecard_data)):
        #add year filter year  to get relevent years
        if farecard_data['To Date'][x][6:10] == "2010" and  int(farecard_data['To Date'][x][0:2]) > 6:
            key1 = "Year -3"
        if farecard_data['To Date'][x][6:10] == "2011" and  int(farecard_data['To Date'][x][0:2]) <= 6:
            key1 = "Year -3"
        if farecard_data['To Date'][x][6:10] == "2011" and  int(farecard_data['To Date'][x][0:2]) > 6:
            key1 = "Year -2"
        if farecard_data['To Date'][x][6:10] == "2012" and  int(farecard_data['To Date'][x][0:2]) <=6:
            key1 = "Year -2"
        if farecard_data['To Date'][x][6:10] == "2012" and int(farecard_data['To Date'][x][0:2]) > 6:
            key1 = "Year -1"
        if farecard_data['To Date'][x][6:10] == "2013" and int(farecard_data['To Date'][x][0:2])<=6:
            key1 = "Year -1"
        if farecard_data['To Date'][x][6:10] == "2013" and  int(farecard_data['To Date'][x][0:2]) > 6:
            key1 = "Year 1"
        if farecard_data['To Date'][x][6:10] == "2014" and int(farecard_data['To Date'][x][0:2])<=6:
            key1 = "Year 1"
        if farecard_data['To Date'][x][6:10] == "2014" and int(farecard_data['To Date'][x][0:2])>6:
            key1 = "Year 2"
        if farecard_data['To Date'][x][6:10] == "2015" and int(farecard_data['To Date'][x][0:2])<=6:
            key1 = "Year 2"
        if farecard_data['To Date'][x][6:10] == "2015" and int(farecard_data["To Date"][x][0:2]) >6:
            key1 = "Year 3"
        if farecard_data['To Date'][x][6:10] == "2016" and int(farecard_data["To Date"][x][0:2]) <=6:
            key1 = "Year 3"
        value1 = int(farecard_data['Full Fare'][x].replace(",", ""))
        value2 =  int(farecard_data['Senior Citizen / Disabled'][x].replace(",", ""))
        value3 = int(farecard_data['7 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
        value4 = int(farecard_data['30 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
        value5 = int(farecard_data['Joint Rail Road Ticket'][x].replace(",", ""))
        value6 =  int(farecard_data['7 Day Unlimited'][x].replace(",", ""))
        value7 =  int(farecard_data['30 Day Unlimited'][x].replace(",", ""))
        value8 =  farecard_data['14 Day Reduced Fare Media Unlimited'][x]
        value9 =  farecard_data['1 Day Unlimited'][x]
        value10 = farecard_data['14 Day Unlimited'][x]
        value11= int(farecard_data['Transit Check Metrocard'][x].replace(",", ""))
        value12 = int(farecard_data['LIB Special Senior'][x].replace(",", ""))
        value13 =  int(farecard_data['Rail Road Unlimited No Trade'][x].replace(",", ""))
        value14 = int(farecard_data['Transit Check Metrocard Annual Metrocard'][x].replace(",", ""))
        value15 =  int(farecard_data['Mail and Ride EZPass Express'][x].replace(",", ""))
        value16 =  int(farecard_data['Mail and Ride Unlimited'][x].replace(",", ""))
        sum_value  = value1 + value2 + value3 + value4 + value5 + value6 + value7 + value8 + value9 + value10 + value11 + value12 + value13 + value14 + value15 + value16
        kv = ("%s\t%s" % (key1, sum_value))
        kv_years.append(kv)
        
kv_years.sort()

keys_years = []
values = []
for i in kv_years:
    key, value = i.split('\t')
    keys_years.append(key)
    value = int(value)
    values.append(value)

key_place = 0
first_key = 0
count = 0
count_place = 0
for j in keys_years:
    if first_key == 0:
        key_place = j
        first_key = 1 
    
    if j != key_place and first_key != 0:
        print("The number of riders in %s were: %d" % (key_place, count))
        key_place = j
        count = values[count_place]
        count_place += 1
    else:
        count += values[count_place]
        count_place += 1

print("The number of riders in %s  were: %d" % (key_place, count))

Farecard Type Breakdown (complete and saved to output files)

In [ ]:
kv_fare_type = []
value_list = []
fare_type_list = ['Full Fare', "Senior Citizen / Disabled", "7 Day ADA Farecard Access System Unlimited", "30 Day ADA Farecard Access System Unlimited", "Joint Rail Road Ticket",
                 "7 Day Unlimited", "30 Day Unlimited", "14 Day Reduced Fare Media Unlimited", "1 Day Unlimited", "14 Day Unlimited", "Transit Check Metrocard", "LIB Special Senior", "Rail Road Unlimited No Trade", "Transit Check Metrocard Annual Metrocard", "Mail and Ride EZPass Express", "Mail and Ride Unlimited"]
value1 = 0 
value2 = 0
value3 = 0
value4 = 0
value5 = 0
value6 = 0
value7 = 0
value8 = 0
value9 = 0
value10 = 0
value11 = 0
value12 = 0
value13 = 0
value14 = 0
value15 = 0
value16 = 0
for x in range(0, len(farecard_data)):
    value1 += int(farecard_data['Full Fare'][x].replace(",", ""))
    value2 +=  int(farecard_data['Senior Citizen / Disabled'][x].replace(",", ""))
    value3 += int(farecard_data['7 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
    value4 += int(farecard_data['30 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
    value5 += int(farecard_data['Joint Rail Road Ticket'][x].replace(",", ""))
    value6 +=  int(farecard_data['7 Day Unlimited'][x].replace(",", ""))
    value7 +=  int(farecard_data['30 Day Unlimited'][x].replace(",", ""))
    value8 +=  farecard_data['14 Day Reduced Fare Media Unlimited'][x]
    value9 +=  farecard_data['1 Day Unlimited'][x]
    value10 += farecard_data['14 Day Unlimited'][x]
    value11 += int(farecard_data['Transit Check Metrocard'][x].replace(",", ""))
    value12 += int(farecard_data['LIB Special Senior'][x].replace(",", ""))
    value13 +=  int(farecard_data['Rail Road Unlimited No Trade'][x].replace(",", ""))
    value14 += int(farecard_data['Transit Check Metrocard Annual Metrocard'][x].replace(",", ""))
    value15 +=  int(farecard_data['Mail and Ride EZPass Express'][x].replace(",", ""))
    value16 +=  int(farecard_data['Mail and Ride Unlimited'][x].replace(",", ""))
sum_value = value1 + value2 + value3 + value4 + value5 + value6 + value7 + value8 + value9 + value10 + value11 + value12 + value13 + value14 + value15 + value16
value_list = [value1, value2, value3, value4, value5, value6, value7, value8, value9, value10, value11, value12, value13, value14, value15, value16]
sum_non_commuter = value1 + value9
count_value = 0
for y in fare_type_list:
    fare_type = y
    total = value_list[count_value]
    percent_of_total = round(float((total/sum_value)*100),2)
    kv_temp = ("There were %d %s rides, which made up %.2f percent of total rides." % (total,fare_type, percent_of_total))
    print(kv_temp)
    count_value += 1
percent_non_comm = round(float((sum_non_commuter / sum_value)*100),2)
print("There were %d rides that had a high likelyhood of not being commuters, representing %.2f of all rides" % (sum_non_commuter, percent_non_comm))

Farecard Type By Station (complete and saved to output files)

In [ ]:
kv_fare_type = []
fare_type_list = ['Full Fare', "Senior Citizen / Disabled", "7 Day ADA Farecard Access System Unlimited", "30 Day ADA Farecard Access System Unlimited", "Joint Rail Road Ticket",
                 "7 Day Unlimited", "30 Day Unlimited", "14 Day Reduced Fare Media Unlimited", "1 Day Unlimited", "14 Day Unlimited", "Transit Check Metrocard", "LIB Special Senior", "Rail Road Unlimited No Trade", "Transit Check Metrocard Annual Metrocard", "Mail and Ride EZPass Express", "Mail and Ride Unlimited"]
def generate(input_list, append_list, fare_type_array, station_key):
    count_place = 0
    for x in range(0, len(fare_type_array)):
        key = fare_type_array[count_place]
        value = input_list[count_place]
        kv = ("%s,%s\t%s" % (station_key, key, value))
        append_list.append(kv)
        count_place += 1
for x in range(0, len(farecard_data)):
    key1 = farecard_data['Station'][x]
    value1 = int(farecard_data['Full Fare'][x].replace(",", ""))
    value2 =  int(farecard_data['Senior Citizen / Disabled'][x].replace(",", ""))
    value3 = int(farecard_data['7 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
    value4 = int(farecard_data['30 Day ADA Farecard Access System Unlimited'][x].replace(",", ""))
    value5 = int(farecard_data['Joint Rail Road Ticket'][x].replace(",", ""))
    value6 =  int(farecard_data['7 Day Unlimited'][x].replace(",", ""))
    value7 =  int(farecard_data['30 Day Unlimited'][x].replace(",", ""))
    value8 =  farecard_data['14 Day Reduced Fare Media Unlimited'][x]
    value9 =  farecard_data['1 Day Unlimited'][x]
    value10 = farecard_data['14 Day Unlimited'][x]
    value11 = int(farecard_data['Transit Check Metrocard'][x].replace(",", ""))
    value12 = int(farecard_data['LIB Special Senior'][x].replace(",", ""))
    value13 =  int(farecard_data['Rail Road Unlimited No Trade'][x].replace(",", ""))
    value14 = int(farecard_data['Transit Check Metrocard Annual Metrocard'][x].replace(",", ""))
    value15 =  int(farecard_data['Mail and Ride EZPass Express'][x].replace(",", ""))
    value16 =  int(farecard_data['Mail and Ride Unlimited'][x].replace(",", ""))
    value_list = [value1, value2, value3, value4, value5, value6, value7, value8, value9, value10, value11, value12, value13, value14, value15, value16]
    generate(value_list, kv_fare_type, fare_type_list, key1)
kv_fare_type.sort()
keys_fare_type = []
values = []
for i in kv_fare_type:
    key, value = i.split('\t')
    keys_fare_type.append(key)
    value = int(value)
    values.append(value)

key_place = 0
first_key = 0
count = 0
count_place = 0
stations_fare_card = []
for j in keys_fare_type:
    if first_key == 0:
        key_place = j
        first_key = 1 
    
    if j != key_place and first_key != 0:
        station_name, fare_card_type = key_place.split(",")
        stations_fare_card.append((station_name, fare_card_type, count))
        #print("The number of %s trips at %s were: %d" % (fare_card_type, station_name, count))
        key_place = j
        count = values[count_place]
        count_place += 1
    else:
        count += values[count_place]
        count_place += 1
station_name, fare_card_type = key_place.split(",")
stations_fare_card.append((station_name, fare_card_type, count))
#print("The number of %s trips at %s  were: %d" % (fare_card_type, station_name, count))

count_vals = 0
count_place = 0
kv_percents = []
temp_list = []
for b in stations_fare_card:
    station_name = b[0]
    fare_card_type = b[1]
    total_rides = b[2]
    temp_list.append((station_name, fare_card_type, total_rides))
    count_vals += 1
    if count_vals == 16:
        sums = 0
        fare_card_temp_list = []
        count_fare_card = 0
        for c in  temp_list:
            sums += c[2]
            fare_card_temp_list.append(c[1])
        if sums == 0:
            sums = 1.0
        else:
            sums = float(sums)
            if c[0] == "103RD ST-ROOSEVELT AVE":
                print(sums)
        for d in temp_list:
            percent_rides = float(round((d[2] / sums)*100,2))
            fare_card_type =  fare_card_temp_list[count_fare_card] 
            kv_percents.append((percent_rides, station_name, fare_card_type))
            count_fare_card += 1
        count_place += 1
        count_vals = 0
        temp_list = []

percent_non_comm = 0
percent_comm = 0
count_vals = 0
diffs_list = []
for z in kv_percents:
    if z[2] == "1 Day Unlimited" or z[2] == "Full Fare":
        percent_non_comm += z[0]
    count_vals += 1
    if count_vals == 16:
        count_vals = 0
        percent_comm = 100.0 - percent_non_comm
        diff_comm = percent_comm - percent_non_comm
        percent_non_comm = 0
        diffs_list.append((diff_comm, z[1]))
#diffs_list.sort(reverse = True)
print(diffs_list)

Weekday Vs Weekend Aggregate Level

In [ ]:
import datetime
kv_weekend = []
count_weektype = []
for x in range(0, len(subway_data)):
    if subway_data['Station'][x] == "Station":
            dummy = 0
    else:
        key1 = subway_data['Station'][x]
        key6 = subway_data['Division'][x]
        key7 = subway_data['Line Name'][x]
        station_name_combo = ("%s%s%s" % (key1, key6, key7))
        ## sample station name clean replacement:
        #if station_name_combo == 'WHITEHALL STBMTR1':
        #        station_name_combo = station_name_combo.replace('WHITEHALL STBMTR1', '1 AVEBMTL')
        
        year = int(subway_data['Date'][x][6:10])
        month = int(subway_data['Date'][x][0:2])
        day = int(subway_data['Date'][x][3:5])
        week_num = datetime.date(year, month, day).weekday()
        if week_num <= 4:
            key5 = "Weekday" 
            date = ("%s-%s-%s" % (month, day, year))
            count_kv_weekday = ("%s,%s" % (key5, date))
            count_weektype.append(count_kv_weekday)
        else:
            key5 = "Weekend"
            date = ("%s-%s-%s" % (month, day, year))
            count_kv_weekend = ("%s,%s" % (key5, date))
            count_weektype.append(count_kv_weekend)
        key2 = subway_data['C/A'][x] 
        key3 = subway_data['Unit'][x]
        key4 = subway_data['SCP'][x]
        key8 = subway_data['Date'][x]
        key9 = subway_data['Time'][x]
        value1 = subway_data['Entries'][x].replace(",", "")
        kv = ("%s&%s&%s&%s^%s^%s\t%s" % (station_name_combo, key2, key3, key4, key8, key9, value1))
        kv_weekend.append(kv)
            
kv_weekend.sort()
count_weektype.sort()

key_place = "hello"
first_key = 0
count_weekday_sum = 0
count_weekend_sum = 0 

for i in count_weektype:
    if first_key == 0:
        key_place = i
        first_key = 1
    if i != key_place and first_key != 0:
        if "Weekend" in key_place:
            count_weekend_sum += 1 
        elif "Weekday" in key_place:
            count_weekday_sum += 1
        else:
            mistake = 1
        key_place = i
    else:
        dummy = 0
if "Weekend" in key_place:
    count_weekend_sum += 1 
elif "Weekday" in key_place:
    count_weekday_sum += 1
keys_weekend = []
dates = []
enter_vals = []
sort_list = []
for i in kv_weekend:
    key_temp, value = i.split('\t')
    key, date, time = key_temp.split("^")
    date_format = datetime.date(int(date[6:10]), int(date[0:2]), int(date[3:5]))
    if time[2] == ":":
        hour =  int(time[0:2])
        minute = int(time[3:5])
        second = int(time[6:8])
    else:
        hour = int(time[0:1])
        minute = int(time[2:4])
        second = int(time[5:7])
    time_format = datetime.time(hour, minute, second)
    key_for_sorting = (key, date_format, time_format, value)
    sort_list.append(key_for_sorting)
sort_list = sorted(sort_list, key = lambda element: (element[0], element[1], element[2]))
for x in sort_list:
    date_string = x[1].isoformat()
    date_string = ("%s-%s-%s" % (date_string[5:7], date_string[8:10], date_string[0:4]))
    new_key = x[0]
    new_value = x[3]
    new_value = int(new_value)
    keys_weekend.append(new_key)
    enter_vals.append(new_value)
    dates.append(date_string)
    

key_place = "hello"
first_key = 0
count_enters_weekend = 0
count_enters_weekday = 0
count_place = 0
count_day = 0
first_count_day = 0
date_stop = len(dates)
new_kv_enter_weekend =[]
new_kv_enter_weekday = []

for j in keys_weekend:
    if first_key == 0:
        key_place = j
        first_key = 1
        date_check = dates[count_place]
    
    if j != key_place and first_key != 0:
        kv_enters_weekend = (count_enters_weekend, key_place)       
        new_kv_enter_weekend.append(kv_enters_weekend) 
        kv_enters_weekday = (count_enters_weekday, key_place)       
        new_kv_enter_weekday.append(kv_enters_weekday)
        key_place = j
        count_init_enters = enter_vals[count_place]
        date_check_old = dates[count_place]
        count_place += 1
        count_day = 0
        count_enters_weekday = 0
        count_enters_weekend = 0
    
    else:
        if first_count_day == 0:
            count_init_enters = enter_vals[count_place]
            first_count_day = 1
        elif count_day <= 1:
            enters = enter_vals[count_place]
            if enters >= count_init_enters:
                count_enters_weekend += (enter_vals[count_place] - count_init_enters)
            else:
                count_enters_weekend += enter_vals[count_place]
        else:
            enters = enter_vals[count_place]
            if enters >= count_init_enters:
                count_enters_weekday += (enter_vals[count_place] - count_init_enters)
            else:
                count_enters_weekday += enter_vals[count_place]
        count_init_enters =  enter_vals[count_place]
        date_check_old = date_check
        count_place += 1
        if count_place < date_stop:
            date_check = dates[count_place]
            if date_check_old != date_check:
                count_day += 1

kv_enters_weekend = (count_enters_weekend, key_place)       
new_kv_enter_weekend.append(kv_enters_weekend) 
kv_enters_weekday = (count_enters_weekday, key_place)       
new_kv_enter_weekday.append(kv_enters_weekday)

            
stations_collapse_weekday = []
stations_collapse_weekend = []
    
def collapse(input_list, append_list, weekend_type):
    for z in input_list:
        station_name = z[1]
        station_sep = z[0]
        collapse_kv = (weekend_type, station_sep)
        append_list.append(collapse_kv) 
collapse(new_kv_enter_weekday, stations_collapse_weekday, "Weekday")
collapse(new_kv_enter_weekend, stations_collapse_weekend, "Weekend")

stations_collapse_weekday.sort()
stations_collapse_weekend.sort()

final_kv_weekday = [] 
final_kv_weekend = []

def aggreg(input_list):
    first_key = 0 
    count_final = 0

    for p in input_list:
        if first_key == 0:
            key_place = p[0]
            first_key = 1
        
        if p[0] != key_place and first_key != 0:
            if "Weekday" in key_place:
                avg_weekday = float(count_final / 5)
                print("The  total number of weekday trips was: %d. The average number of trips on a weekday was: %.2f." % (count_final, avg_weekday))
            else: 
                avg_weekend = float(count_final / 2)
                print("The total number of weekend trips was: %d. The average number of trips on a weekend was: %.2f." % (count_final, avg_weekend))
            count_final = p[1]
            key_place = p[0]
        else:
            count_final += p[1]
    if "Weekday" in key_place:
        avg_weekday = float(count_final / 5)
        print("The total number of weekday trips was %d. The average number of trips on a weekday was: %.2f." % (count_final, avg_weekday))
    else: 
        avg_weekend = float(count_final / 2)
        print("The total number of weekend trips was: %d. The average number of trips on a weekend was: %.2f." % (count_final, avg_weekend))
aggreg(stations_collapse_weekday) 
aggreg(stations_collapse_weekend)
print("The Total Number of Weekdays was: %d. The Total Number of Weekends  was %d." % (count_weekday_sum, count_weekend_sum))

Weekend Vs Weekday Station Level

In [ ]:
import datetime
kv_weekend = []
count_weektype = []
for x in range(0, len(subway_data)):
    if subway_data['Station'][x] == "Station":
            dummy = 0
    else:
        key1 = subway_data['Station'][x]
        key6 = subway_data['Division'][x]
        key7 = subway_data['Line Name'][x]
        station_name_combo = ("%s%s%s" % (key1, key6, key7))
        ## sample station name clean replacement:
        #if station_name_combo == 'WHITEHALL STBMTR1':
        #        station_name_combo = station_name_combo.replace('WHITEHALL STBMTR1', '1 AVEBMTL')
        
        year = int(subway_data['Date'][x][6:10])
        month = int(subway_data['Date'][x][0:2])
        day = int(subway_data['Date'][x][3:5])
        week_num = datetime.date(year, month, day).weekday()
        if week_num <= 4:
            key5 = "Weekday" 
            date = ("%s-%s-%s" % (month, day, year))
            count_kv_weekday = ("%s,%s" % (key5, date))
            count_weektype.append(count_kv_weekday)
        else:
            key5 = "Weekend"
            date = ("%s-%s-%s" % (month, day, year))
            count_kv_weekend = ("%s,%s" % (key5, date))
            count_weektype.append(count_kv_weekend)
        key2 = subway_data['C/A'][x] 
        key3 = subway_data['Unit'][x]
        key4 = subway_data['SCP'][x]
        key8 = subway_data['Date'][x]
        key9 = subway_data['Time'][x]
        value1 = subway_data['Entries'][x].replace(",", "")
        kv = ("%s&%s&%s&%s^%s^%s\t%s" % (station_name_combo, key2, key3, key4, key8, key9, value1))
        kv_weekend.append(kv)
            
kv_weekend.sort()
count_weektype.sort()

key_place = "hello"
first_key = 0
count_weekday_sum = 0
count_weekend_sum = 0 

for i in count_weektype:
    if first_key == 0:
        key_place = i
        first_key = 1
    if i != key_place and first_key != 0:
        if "Weekend" in key_place:
            count_weekend_sum += 1 
        elif "Weekday" in key_place:
            count_weekday_sum += 1
        else:
            mistake = 1
        key_place = i
    else:
        dummy = 0
if "Weekend" in key_place:
    count_weekend_sum += 1 
elif "Weekday" in key_place:
    count_weekday_sum += 1
keys_weekend = []
enter_vals = []
dates = []
sort_list = []
for i in kv_weekend:
    key_temp, value = i.split('\t')
    key, date, time = key_temp.split("^")
    date_format = datetime.date(int(date[6:10]), int(date[0:2]), int(date[3:5]))
    if time[2] == ":":
        hour =  int(time[0:2])
        minute = int(time[3:5])
        second = int(time[6:8])
    else:
        hour = int(time[0:1])
        minute = int(time[2:4])
        second = int(time[5:7])
    time_format = datetime.time(hour, minute, second)
    key_for_sorting = (key, date_format, time_format, value)
    sort_list.append(key_for_sorting)
sort_list = sorted(sort_list, key = lambda element: (element[0], element[1], element[2]))
for x in sort_list:
    date_string = x[1].isoformat()
    date_string = ("%s-%s-%s" % (date_string[5:7], date_string[8:10], date_string[0:4]))
    new_key = x[0]
    new_value = x[3]
    new_value = int(new_value)
    keys_weekend.append(new_key)
    enter_vals.append(new_value)
    dates.append(date_string)
    
key_place = "hello"
first_key = 0
count_enters_weekend = 0
count_enters_weekday = 0
count_place = 0
count_day = 0
first_count_day = 0
count_time_per = 0
new_kv_enter_weekend =[]
new_kv_enter_weekday = []
date_stop = len(dates)

for j in keys_weekend:
    if first_key == 0:
        key_place = j
        first_key = 1
        date_check = dates[count_place]
    
    if j != key_place and first_key != 0:
        kv_enters_weekend = (count_enters_weekend, key_place)       
        new_kv_enter_weekend.append(kv_enters_weekend) 
        kv_enters_weekday = (count_enters_weekday, key_place)       
        new_kv_enter_weekday.append(kv_enters_weekday)
        key_place = j
        count_init_enters = enter_vals[count_place]
        date_check_old = dates[count_place]
        count_place += 1
        count_day = 0
        count_enters_weekday = 0
        count_enters_weekend = 0
    
    else:
        if first_count_day == 0:
            count_init_enters = enter_vals[count_place]
            first_count_day = 1
        elif count_day <= 1:
            enters = enter_vals[count_place]
            if enters >= count_init_enters:
                count_enters_weekend += (enter_vals[count_place] - count_init_enters)
            else:
                count_enters_weekend += enter_vals[count_place]
        else:
            enters = enter_vals[count_place]
            if enters >= count_init_enters:
                count_enters_weekday += (enter_vals[count_place] - count_init_enters)
            else:
                count_enters_weekday += enter_vals[count_place]
        count_init_enters =  enter_vals[count_place]
        date_check_old = date_check
        count_place += 1
        if count_place < date_stop:
            date_check = dates[count_place]
            if date_check_old != date_check:
                count_day += 1
        
kv_enters_weekend = (count_enters_weekend, key_place)       
new_kv_enter_weekend.append(kv_enters_weekend) 
kv_enters_weekday = (count_enters_weekday, key_place)       
new_kv_enter_weekday.append(kv_enters_weekday)

            
stations_collapse_weekday = []
stations_collapse_weekend = []

def collapse(input_list, append_list, weekend_type):
    for z in input_list:
        station_name = z[1] 
        station_name_only, dummy1, dummy2, dummy3 = station_name.split("&")
        station_sep = z[0]
        collapse_kv = (("%s@%s" % (station_name_only, weekend_type)), station_sep)
        append_list.append(collapse_kv)
collapse(new_kv_enter_weekday, stations_collapse_weekday, "Weekday")
collapse(new_kv_enter_weekend, stations_collapse_weekend, "Weekend")

new_kv_enter_both = []
for n in stations_collapse_weekday:
    new_kv_enter_both.append(n)
for m in stations_collapse_weekend:
    new_kv_enter_both.append(m)

new_kv_enter_both.sort()
final_kv_weekday = [] 
final_kv_weekend = [] 

diff_list = [] 
def aggreg(input_list, append_list):
    first_key = 0 
    count_final = 0
    avg_weekday = 11111111.0
    avg_weekend = 11111111.0
    for p in input_list:
        if first_key == 0:
            key_place = p[0]
            first_key = 1
        
        if p[0] != key_place and first_key != 0:
            if week_type == "Weekday":
                avg_weekday = float(count_final / 5)
                print("The total number of weekday trips at %s was: %d. The average number of trips on a weekday was: %.2f." % (station_name, count_final, avg_weekday))
            else: 
                avg_weekend = float(count_final / 2)
                print("The total number of weekend trips at %s was: %d. The average number of trips on a weekend was: %.2f." % (station_name, count_final, avg_weekend))
            if avg_weekday == 11111111.0 or avg_weekend == 11111111.0:
                dummy = 0
            else:
                diff_avg = avg_weekday - avg_weekend
                diff_kv = (diff_avg, station_name)
                append_list.append(diff_kv)
                avg_weekday = 11111111.0
                avg_weekend = 11111111.0
            count_final = p[1]
            key_place = p[0]
            station_name, week_type  =  p[0].split("@")
        else:
            count_final += p[1]
            station_name, week_type  =  p[0].split("@")
    if week_type == "Weekday":
        avg_weekday = float(count_final / 5)
        print("The total number of weekday trips  at %s was %d. The average number of trips on a weekday was: %.2f." % (station_name, count_final, avg_weekday))
    else: 
        avg_weekend = float(count_final / 2)
        print("The total number of weekend trips at %s was: %d. The average number of trips on a weekend was: %.2f." % (station_name, count_final, avg_weekend))
    diff_avg = avg_weekday - avg_weekend
    diff_kv = (diff_avg, station_name)
    append_list.append(diff_kv)
aggreg(new_kv_enter_both, diff_list)

def ranking(array, order, top_or_bottom):
    array.sort(reverse = order)
    top_ten = []
    bottom_ten = []
    count_place = 1
    for x in range(0,1):
        top_ten.append(array[x])
    for y in top_ten:
        key_final = y[1]
        value_final = y[0]
        print("The %d station with the %s weekday to weekend average trip differential is: %s with a %.2f differential" % (count_place, top_or_bottom, key_final, value_final))
        count_place +=1
ranking(diff_list, True, "biggest")
ranking(diff_list, False, "smallest")
print("The Total Number of Weekday Trips was: %.2f. The Total Number of Weekend Trips was %.2f." % (count_weekday_sum, count_weekend_sum))

Rides By Day Aggregate

In [ ]:
import datetime
kv_days = []

for x in range(0, len(subway_data)):
    if subway_data['Station'][x] == "Station":
            dummy = 0
    else:
        key1 = subway_data['Station'][x]
        key5 = subway_data['Division'][x]
        key6 = subway_data['Line Name'][x]
        station_name_combo = ("%s%s%s" % (key1, key5, key6))
        ## sample station name clean replacement:
        #if station_name_combo == 'WHITEHALL STBMTR1':
        #        station_name_combo = station_name_combo.replace('WHITEHALL STBMTR1', '1 AVEBMTL')
        key2 = subway_data['C/A'][x] 
        key3 = subway_data['Unit'][x]
        key4 = subway_data['SCP'][x]
        key7 = subway_data['Date'][x]
        key8 = subway_data['Time'][x]
        value1 = subway_data['Entries'][x].replace(",", "")
        kv = ("%s&%s&%s&%s^%s^%s\t%s" % (station_name_combo, key2, key3, key4, key7, key8, value1))
        kv_days.append(kv)
            
kv_days.sort()


keys_days = []
enter_vals = []
sort_list = []

for i in kv_days:
    key_temp, value = i.split('\t')
    key, date, time = key_temp.split("^")
    date_format = datetime.date(int(date[6:10]), int(date[0:2]), int(date[3:5]))
    if time[2] == ":":
        hour =  int(time[0:2])
        minute = int(time[3:5])
        second = int(time[6:8])
    else:
        hour = int(time[0:1])
        minute = int(time[2:4])
        second = int(time[5:7])
    time_format = datetime.time(hour, minute, second)
    key_for_sorting = (key, date_format, time_format, value)
    sort_list.append(key_for_sorting)

sort_list = sorted(sort_list, key = lambda element: (element[0], element[1], element[2]))
for x in sort_list:
    date_string = x[1].isoformat()
    date_string = ("%s-%s-%s" % (date_string[5:7], date_string[8:10], date_string[0:4]))
    new_key = ("%s&%s" % (x[0], date_string))
    new_value = x[3]
    new_value = int(new_value)
    keys_days.append(new_key)
    enter_vals.append(new_value)

key_place = "hello"
first_key = 0
count_enters_days = 0 
count_place = 0
first_count_day = 0
count_time_per = 0
new_kv_enter_days = []


for j in keys_days:
    if first_key == 0:
        key_place = j
        first_key = 1 
    
    if j != key_place and first_key != 0:
        station_name, dummy1, dummy2, dummy3, date = key_place.split("&")
        kv_enters_days = (count_enters_days, station_name, date)       
        new_kv_enter_days.append(kv_enters_days)
        key_place = j
        count_init_enters = enter_vals[count_place]
        count_place += 1
        count_enters_days = 0
    
    else: 
        if first_count_day == 0:
                count_init_enters = enter_vals[count_place]
                count_place += 1
                first_count_day = 1
        else:
            enters = enter_vals[count_place]
            if enters >= count_init_enters:
                count_enters_days += (enter_vals[count_place] - count_init_enters)
                count_init_enters = enter_vals[count_place]
            else:
                count_enters_days += enter_vals[count_place]
                count_init_enters = enter_vals[count_place]
            count_place += 1
        
station_name, dummy1, dummy2, dummy3, date = key_place.split("&")
kv_enters_days = (count_enters_days, station_name, date)       
new_kv_enter_days.append(kv_enters_days)

            
stations_collapse_days = []
    
def collapse(input_list, append_list):
    for z in input_list:
        station_name = z[1]
        station_sep = z[0]
        date = z[2]
        collapse_kv = (date, station_sep)
        append_list.append(collapse_kv)
collapse(new_kv_enter_days, stations_collapse_days)

stations_collapse_days.sort()

final_kv_days = []

def aggreg(input_list):
    first_key = 0 
    count_final = 0

    for p in input_list:
        if first_key == 0:
            key_place = p[0]
            first_key = 1
        
        if p[0] != key_place and first_key != 0:
            print("The number of trips on %s was: %d" % (key_place, count_final))
            final_kv = (key_place, count_final)
            final_kv_days.append(final_kv)
            count_final = p[1]
            key_place = p[0]
        else:
            count_final += p[1]
    final_kv = (key_place, count_final)
    final_kv_days.append(final_kv)
    print("The number of trips on %s was: %d" % (key_place, count_final))
aggreg(stations_collapse_days)

Plots

In [ ]:
import matplotlib.pyplot as plt

def plot(input_list, title, ylabel, xlabel, Column1, Column2, file_name, xticks_yes):
    days_tick_list = ['Year 1', 'Year 2', 'Year 3']
    x_list = []
    y_list = []
    for x in range(0, len(input_list)):
        temp = input_list[Column1][x]
        temp1 = int(input_list[Column2][x])
        temp = temp.strip("'")
        temp = temp.strip(" ")
        temp = temp.strip("'")
        x_list.append(temp)
        y_list.append(temp1)


    N = len(y_list)
    x = range(N)
    plt.figure(figsize=(55,40))
    plt.title(title,fontsize=70)
    plt.bar(x, y_list, color="b", align="center",width=0.3)
    plt.tick_params(axis = 'both', which = 'major', labelsize = 30)
    if xticks_yes == 1:
        plt.xticks([15, 365, 730], days_tick_list[0:],rotation='horizontal',fontsize=60)
    else:
        plt.xticks(range(len(x_list)), x_list[0:],rotation='vertical',fontsize=30)
    plt.ylabel(ylabel,fontsize=45) 
    plt.xlabel(xlabel, fontsize = 45) 
    plt.savefig('C:/Users/Benjamin/Desktop/BigData1004/Term Project/Graphs/'+file_name+'.png', bbox_inches= 'tight')
    plt.close('all')
    #plt.show()
#plot(days_graph, "Rides Per Day", "Number of Rides", "Year", "Days", "Counts", "citibike_day_rides", 1)
plot(top_20_routes_graph, "Top 20 Subway Stations", "Number of Rides", "Stations", "Station", "Number of Trips", "subway_top_20_stations_farecard", 0)
plot(farecard_type_station_graph, "Commuter Vs Non-Commuter Differentials By Station", "Percent Difference", "Station", "Station", "Commute To Non Commute Differential", "subway_farecard_type_stations_differential",0)
#plot(top_10_ends_stations_graph, "Top 10 Ending Stations", "Number of Rides", "End Station", "Station Name", "Counts", "citibike_top_10_ends_stations", 0)

In [ ]:
hjs \
-files /home/bb1976/project/subway_trips_by_day \
-D mapreduce.job.reduces=1 \
-mapper subway_trips_by_day/mapper.sh \
-reducer subway_trips_by_day/reducer.sh \
-input /user/bb1976/subway_data_file.csv \
-output /user/bb1976/subway_trips_by_day.out



. /etc/profile.d/modules.sh
module load python/gnu/3.4.4
directoryXXX/map.py
